**bold text**##Setup

In [1]:
from IPython.utils import io

with io.capture_output() as captured:
  !pip install folium
print("Installation 1 Complete")

with io.capture_output() as captured:
  !pip install openrouteservice
print("Installation 2 Complete")

Installation 1 Complete
Installation 2 Complete


In [2]:
import openrouteservice as ors
import folium
import numpy as np
import pandas as pd

from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
geocoder  = Nominatim(user_agent="My App")
import re

# Adding 1 second padding between calls to server
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geocoder.geocode, min_delay_seconds = 1,   return_value_on_exception = None) 

#Connect to Google Sheets
from gspread import authorize
from gspread.models import Cell
from oauth2client.service_account import ServiceAccountCredentials
from google.colab import drive
drive.mount("/content/drive")
scopes = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]

cred = ServiceAccountCredentials.from_json_keyfile_name("/content/drive/My Drive/Bread of the Mighty Project/credentials.json", scopes)
gclient = authorize(cred)

# Define lat and long functions
def find_loc(address):
  address = re.sub(" Ste \d+", "", address) #remove suite 
  #address = re.sub(",.*,", "", address) #remove city
  if (address == 'Missing'):
    location = 'No address input'   
  else:    
    try: location = geocoder.geocode(address)
    except GeocoderTimedOut as e: location = "Timeout Error"
  if location is None: 
    location = 'Not found by geopy'   
  return location

def coords(key):
    return (find_long[key], find_lat[key])

def find_lat(address):
  location = find_loc(address)
  if isinstance(location, str): return location
  else: return round(location.latitude,6)

def find_long(address):
  location = find_loc(address)
  if isinstance(location, str): return location
  else: return round(location.longitude,6)


Mounted at /content/drive


In [3]:
# Add Coordinates for New Locations (~1 sec / new location)

# Access Google Sheet
worksheet = gclient.open("BOTM Database").sheet1
rows = worksheet.get_all_values()

# Convert to a DataFrame
columns = rows[0]
df = pd.DataFrame.from_records(rows[1:], columns=columns) 
new_df = df[(df['LAT'] == '') | (df['LONG'] == '')]

# Calc missing coordinates
cell_list = []
lat_col = df.columns.get_loc('LAT') + 1
long_col = df.columns.get_loc('LONG') + 1
for index, row in new_df.iterrows():
  #df['LAT'] = df.apply(lambda row: find_lat(row['ADDRESS']) if row['LAT'] == '' else row['LAT'], axis=1)
  cell_list.append(Cell(row=index+2, col=lat_col, value=find_lat(row['ADDRESS'])))
  cell_list.append(Cell(row=index+2, col=long_col, value=find_long(row['ADDRESS'])))

if cell_list != []: worksheet.update_cells(cell_list)

## New (Optimized) Routes

In [4]:
from os import truncate
from folium.plugins import BeautifyIcon

# Access Google Sheet
worksheet = gclient.open("BOTM Database").sheet1
rows = worksheet.get_all_values()
columns = rows[0]
df = pd.DataFrame.from_records(rows[1:], columns=columns) 

#filter by numerical lat/long and day of the week
df = df[df['LAT'].apply(lambda x: not re.search("[A-Za-z]", x))]
df = df[df['LONG'].apply(lambda x: not re.search("[A-Za-z]", x))]
df = df[(df['LAT'] != '') | (df['LONG'] != '')]

df_mon = df[df['MON']!=''].sort_values(by=['MON'])
df_tues = df[df['TUES']!=''].sort_values(by=['TUES'])
df_wed = df[df['WED']!=''].sort_values(by=['WED'])
df_thurs = df[df['THURS']!=''].sort_values(by=['THURS'])
df_fri = df[df['FRI']!=''].sort_values(by=['FRI'])
df_week = [df_mon, df_tues, df_wed, df_thurs, df_fri]
days = {'M': 0, 'T': 1, 'W': 2, 'R': 3, 'F': 4}

from pandas._libs.hashtable import value_count
    
# The vehicles are all located at BOTM
depot = [29.660709,-82.328536]

# Define the vehicles
# https://openrouteservice-py.readthedocs.io/en/latest/openrouteservice.html#openrouteservice.optimization.Vehicle
vehicles = []
for idx in range(3): #Number of vehicles
    vehicles.append(
        ors.optimization.Vehicle(
            id=idx, 
            start=list(reversed(depot)),
            end=list(reversed(depot)),
            capacity=[5000],
            time_window=[1553241600, 1553284800]  # Fri 8-20:00, expressed in POSIX timestamp
        )
    )
    
# Next define the delivery stations
# # https://openrouteservice-py.readthedocs.io/en/latest/openrouteservice.html#openrouteservice.optimization.Job

deliveries_week = []
for df in df_week:
  deliveries = []
  deliveries_week.append(deliveries)
  for delivery in df.itertuples():
      loc_long = float(delivery.LONG)
      loc_lat = float(delivery.LAT)
      deliveries.append(
          ors.optimization.Job(
              id=delivery.Index,
              location=[loc_long, loc_lat], #REVERSED
              service= 20*60,  # Assume 20 minutes at each site
              #amount = [50],  
              amount= [int(delivery.AMOUNT)],
              #time_windows=[[
                  #int(delivery.Open_From.timestamp()),  # VROOM expects UNIX timestamp
                  #int(delivery.Open_To.timestamp())
              #]]
        ))

In [6]:
# Define the map centered around BOTM

#deliveries = deliveries_week[0]

f_all = []

days_list = list(days.keys())

for day_ind in range(len(days_list)):
  deliveries_today = deliveries_week[day_ind]
  df_today = df_week[day_ind]

  f = folium.Figure(height=500, width=1000)
  m = folium.Map(location=[29.616321, -82.422549], zoom_start=10).add_to(f)

  folium.Marker(
          location=depot,
          icon=folium.Icon(color="green", icon="bus", prefix='fa')#,
          #setZIndexOffset=1000
          ).add_to(m)

  # Plot the locations on the map with more info in the ToolTip
  for index, row in df_today.iterrows():
      lat = float(row['LAT'])
      long = float(row['LONG'])

      tooltip = (folium.Tooltip(
          """<h5>{LOCATION} / {NOTES} </h5>
          <b>Address:</b> {ADDRESS} <br>
          <b>Location:</b> ({LAT}, {LONG}) <br>
          """.format(**row)))
      
      folium.Marker(
          location=(lat, long),
          tooltip=tooltip
          #popup=row['LOCATION']
          ).add_to(m)

      # icon=BeautifyIcon(
      #             icon_shape='marker',
      #             number=index, #int(location.Index)
      #             spin=True,
      #             text_color='red',
      #             background_color="#FFF",
      #             inner_icon_style="font-size:12px;padding-top:-5px;"
      #             ).add_to(m)

  # Initialize a client and make the request
  #ors_client = ors.Client(base_url='http://localhost:8080/ors')
  ors_client = ors.Client(key='5b3ce3597851110001cf62489217d34c333d469aa2034e367d70cb0a')  # Get an API key from https://openrouteservice.org/dev/#/signup
  result = ors_client.optimization(
      jobs=deliveries_today,
      vehicles=vehicles,
      geometry=True,
  )

  # Add the output to the map
  for color, route in zip(['green', 'red', 'blue'], result['routes']):
      decoded = ors.convert.decode_polyline(route['geometry'])  # Route geometry is encoded
      gj = folium.GeoJson(
          name='Vehicle {}'.format(route['vehicle']),
          data={"type": "FeatureCollection", "features": [{"type": "Feature",
                                                          "geometry": decoded,
                                                          "properties": {"color": color}
                                                          }]},
          style_function=lambda x: {"color": x['properties']['color']}
      )
      
      route["miles"] = round(route["distance"] / 1000 * 0.621371, 1)
      route["hrs"] = route["duration"] // 3600
      route["mins"] = round((route["duration"] % 3600) / 60)

      gj.add_child(folium.Tooltip(
          """<h5>Vehicle {vehicle}</h5>
          <b>Distance:</b> {miles} mi <br>
          <b>Duration:</b> {hrs} hr {mins} min
          """.format(**route)
      ))
      gj.add_to(m)

  folium.LayerControl().add_to(m)
  f_all.append(f)
  #print(route[Vehicle_0])

In [8]:
f_all[days['R']]

In [ ]:
print(route["miles"])

4.2


In [ ]:
#average time for capacity 5000
Vehicle1T = 52
Vehicle2T = 26
Vehicle3T = 143
avgDuration = (Vehicle1T + Vehicle2T + Vehicle3T)/3
print(avgDuration)

73.66666666666667


##Current Routes

In [ ]:
#CURRENT BOTM ROUTES
#tooltips/icons
#comapare times/distances
#loop through for different days

f_current = []

f = folium.Figure(height=500, width=1000)
map_mon = folium.Map(location=[29.616321, -82.422549], zoom_start=10).add_to(f)

# for index, row in df_mon.iterrows():
#       lat = float(row['LAT'])
#       long = float(row['LONG'])

#       tooltip = (folium.Tooltip(
#           """<h5>{LOCATION} / {NOTES} </h5>
#           <b>Address:</b> {ADDRESS} <br>
#           <b>Location:</b> ({LAT}, {LONG}) <br>
#           """.format(**row)))
      
#       folium.Marker(
#           location=(lat, long),
#           tooltip=tooltip
#           #popup=row['LOCATION']
#           ).add_to(map_mon)

folium.Marker(
          location=depot,
          icon=folium.Icon(color="green", icon="bus", prefix='fa')#,
          #setZIndexOffset=1000
          ).add_to(map_mon)


for index, row in df_mon.iterrows():
      coords = (float(row['LONG']), float(row['LAT']))

coords_mon = {}

for index, row in df_mon.iterrows():
  vehicle = row['MON'][0] #vehicle id (a, b, c, etc.)
  coords = (float(row['LONG']), float(row['LAT']))

  if vehicle not in coords_mon.keys():
    coords_mon[vehicle] = []
    coords_mon[vehicle].append((-82.328536, 29.660709)) #start at botm
    
  coords_mon[vehicle].append(coords)

  if '*' in row['MON']:
    coords_mon[vehicle].append((-82.328536, 29.660709)) #add botm stop

routes_mon = []

for veh in coords_mon:
    coords_mon[veh].append((-82.328536, 29.660709)) #end at botm

    finished = (len(coords_mon[veh]) >= 4) #need at least 4 sets of coords
    while not finished:
       coords_mon[veh].append((-82.328536, 29.660709))
       finished = (len(coords_mon[veh]) >= 4)

    route = ors_client.directions(coordinates=coords_mon[veh], profile='driving-car', format='geojson', optimize_waypoints=False)
    routes_mon.append(route)

veh_code = {'a': 0, 'b': 1, 'c': 2, 'd':3}

def style_function(color):
    return lambda feature: dict(color=color, weight=3, opacity=1)

colors_list = ['orange', 'blue', 'red', 'green', 'black', 'purple']
  
for i in range(len(coords_mon)):
    veh = list(veh_code.keys())[i]
    color = colors_list[i]
  
    gj = folium.GeoJson(routes_mon[veh_code[veh]], name=veh, style_function=style_function(color))
   
    # route["miles"] = round(route["distance"] / 1000 * 0.621371, 1)
    # route["hrs"] = route["duration"] // 3600
    # route["mins"] = round((route["duration"] % 3600) / 60)

    # gj.add_child(folium.Tooltip(
    #       """<h5>Vehicle {vehicle}</h5>
    #       <b>Distance:</b> {miles} mi <br>
    #       <b>Duration:</b> {hrs} hr {mins} min
    #       """.format(**route)
    #   ))
    gj.add_to(map_mon)


folium.LayerControl().add_to(map_mon)
f_current.append(f)

f_current[days['M']]

In [ ]:
#CURRENT BOTM ROUTES - TUES
#tooltips/icons
#comapare times/distances
#loop through for different days

f = folium.Figure(height=500, width=1000)
map_tues = folium.Map(location=[29.616321, -82.422549], zoom_start=10).add_to(f)

# for index, row in df_tues.iterrows():
#       lat = float(row['LAT'])
#       long = float(row['LONG'])

#       tooltip = (folium.Tooltip(
#           """<h5>{LOCATION} / {NOTES} </h5>
#           <b>Address:</b> {ADDRESS} <br>
#           <b>Location:</b> ({LAT}, {LONG}) <br>
#           """.format(**row)))
      
#       folium.Marker(
#           location=(lat, long),
#           tooltip=tooltip
#           #popup=row['LOCATION']
#           ).add_to(map_tues)

folium.Marker(
          location=depot,
          icon=folium.Icon(color="green", icon="bus", prefix='fa')#,
          #setZIndexOffset=1000
          ).add_to(map_tues)


for index, row in df_tues.iterrows():
      coords = (float(row['LONG']), float(row['LAT']))

coords_tues = {}

for index, row in df_tues.iterrows():
  vehicle = row['TUES'][0] #vehicle id (a, b, c, etc.)
  coords = (float(row['LONG']), float(row['LAT']))

  if vehicle not in coords_tues.keys():
    coords_tues[vehicle] = []
    coords_tues[vehicle].append((-82.328536, 29.660709)) #start at botm
    
  coords_tues[vehicle].append(coords)

  if '*' in row['TUES']:
    coords_tues[vehicle].append((-82.328536, 29.660709)) #add botm stop

routes_tues = []

for veh in coords_tues:
    coords_tues[veh].append((-82.328536, 29.660709)) #end at botm

    finished = (len(coords_tues[veh]) >= 4) #need at least 4 sets of coords
    while not finished:
       coords_tues[veh].append((-82.328536, 29.660709))
       finished = (len(coords_tues[veh]) >= 4)

    route = ors_client.directions(coordinates=coords_tues[veh], profile='driving-car', format='geojson', optimize_waypoints=False)
    routes_tues.append(route)

veh_code = {'a': 0, 'b': 1, 'c': 2, 'd':3}

def style_function(color):
    return lambda feature: dict(color=color, weight=3, opacity=1)

colors_list = ['blue', 'red', 'green', 'orange', 'black', 'purple']
  
for i in range(len(coords_tues)):
    veh = list(veh_code.keys())[i]
    color = colors_list[i]
  
    gj = folium.GeoJson(routes_tues[veh_code[veh]], name=veh, style_function=style_function(color))
   
    # route["miles"] = round(route["distance"] / 1000 * 0.621371, 1)
    # route["hrs"] = route["duration"] // 3600
    # route["mins"] = round((route["duration"] % 3600) / 60)

    # gj.add_child(folium.Tooltip(
    #       """<h5>Vehicle {vehicle}</h5>
    #       <b>Distance:</b> {miles} mi <br>
    #       <b>Duration:</b> {hrs} hr {mins} min
    #       """.format(**route)
    #   ))
    gj.add_to(map_tues)


folium.LayerControl().add_to(map_tues)
f_current.append(f)

f_current[days['T']]

In [ ]:
routes_tues

[{'bbox': [-82.389024, 29.652048, -82.328376, 29.71145],
  'features': [{'bbox': [-82.389024, 29.652048, -82.328376, 29.71145],
    'geometry': {'coordinates': [[-82.328376, 29.660625],
      [-82.328518, 29.66042],
      [-82.328593, 29.660304],
      [-82.32862, 29.660271],
      [-82.328637, 29.660251],
      [-82.328669, 29.660223],
      [-82.328749, 29.660197],
      [-82.328819, 29.660194],
      [-82.328829, 29.659878],
      [-82.328822, 29.65953],
      [-82.328811, 29.659418],
      [-82.328905, 29.659474],
      [-82.329038, 29.659476],
      [-82.329186, 29.659429],
      [-82.329795, 29.659428],
      [-82.329955, 29.659427],
      [-82.330144, 29.659427],
      [-82.330692, 29.659425],
      [-82.330745, 29.659421],
      [-82.330875, 29.659413],
      [-82.330876, 29.659301],
      [-82.330883, 29.658774],
      [-82.330886, 29.658473],
      [-82.330869, 29.658105],
      [-82.330841, 29.657524],
      [-82.330835, 29.657476],
      [-82.33083, 29.657409],
      [-82.3